### Input data
Csv file in the format
```
program, method, memory size, explored, optimum, time, design space size
```
that contains the results of runnig PAYNT for a set amount of time for each program and setup (using the basic Iterative Approach, Memory Injection Approach, and Incremental restrictions Approach).

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import numpy as np


out = pd.read_csv("../workspace/log_old/output.csv", sep=",",
                  names=["program", "Approach", "method", "memory_size", "explored",
                         "optimum", "time", "design_space_size", "iterations"],
                  )
out["optimum"] = pd.to_numeric(out["optimum"], errors="coerce")
out["time"] = pd.to_numeric(out["time"], errors="coerce")
out["memory_size"] = pd.to_numeric(out["memory_size"], errors="coerce")
out = out.where(out["time"] > 0).dropna()
out["Approach"] = out["Approach"].replace("Full incremental", "Incremental")
out = out.where(out["memory_size"] > 1).dropna()
copy = out

out.where((out["program"].str.contains("maze_long")) ).dropna()

In [ ]:
group = copy.groupby(["program","memory_size", "method", ]).last().reset_index()
group

In [ ]:
for program in group["program"].unique():
    f = open("tables/table_{}.tex".format(program[25:-13].replace("/", "_")), "w")
    f.write(
        group.where(group["program"] == program).dropna()[["method","memory_size", "optimum", "time"]].to_latex(index=False)
    )


In [ ]:
grouped = out.where((out["program"].str.contains("grid")) & (out["program"].str.contains("simple") == False)).dropna().groupby([
    "program", "Approach", "memory_size"
]).agg({"optimum": "max", "time": "sum"})

grouped = grouped.append(
    out.where(
        (out["program"].str.contains("grid")) & (out["program"].str.contains("simple")) |
        (out["program"].str.contains("maze"))
    ).dropna().groupby([
        "program", "Approach", "memory_size"
    ]).agg({"optimum": "min", "time": "sum"})
)
grouped = grouped.reset_index()
grouped

In [ ]:
sns.set(font_scale = 1.2, style="whitegrid")

In [ ]:
for program in grouped["program"].unique():
    grid = sns.relplot(
                data=grouped.where(grouped["program"] == program), #.dropna(),
                x="memory_size",
                y="optimum",
                kind="line",
                hue="Approach",
                hue_order=["Incremental", "Injection", "Iterative"],
                legend="full",
                height=5,
                aspect=1.8,
                facet_kws={'sharey': False, 'sharex': False},
                alpha=0.7,
            )
    grid.set(title=program[25:-13].replace("/", " ").replace("_", " ").capitalize())
    grid.set_xlabels("Memory size")
    grid.set_ylabels("Optimum")
    grid.savefig("../workspace/log/" +
                 program[25:-13].replace("/", "_") + "_mem_x_opt.png")


In [ ]:
out["time_sum"] = out.groupby(["program", "Approach"]).time.transform(pd.Series.cumsum)

for program in out["program"].unique():
    grid = sns.relplot(
                data=out.where(out["program"] == program), 
                x="time_sum",
                y="optimum",
                kind="line",
                hue="Approach",
                hue_order=["Incremental", "Injection", "Iterative"],
                legend="full",
                height=5,
                aspect=1.8,
                facet_kws={'sharey': False, 'sharex': False},
                alpha=0.7,
            )
    grid.set(xscale="log")

    grid.set(title=program[25:-13].replace("/", " ").replace("_", " ").capitalize())
    grid.set_xlabels("Time [s]")
    grid.set_ylabels("Optimum")
    for ax in grid.axes.flat:
        ax.xaxis.set_major_formatter(FuncFormatter(lambda y, _: '{:.16g}'.format(y)))
    grid.savefig("../workspace/log/" +
                 program[25:-13].replace("/", "_") + "_time_x_opt.png")
